In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd
import sys
import networkx as nx
import os
import warnings
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Experiment Graph
from execution_environment import ExecutionEnvironment as ee

# Suppress warnings
warnings.filterwarnings('ignore')
% matplotlib inline


try:
    import pygraphviz
    from networkx.drawing.nx_agraph import graphviz_layout
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import graphviz_layout
    except ImportError:
        raise ImportError("This example needs Graphviz and either "
                          "PyGraphviz or pydot")
        
def plot_graph():
    plt.figure(figsize=(12, 12))
    #pos = nx.spring_layout(ee.graph.graph)
    pos = graphviz_layout(ee.graph.graph, prog='twopi', args='')
    color_map = []
    for node in ee.graph.graph.nodes(data=True):
        if node[1]['root']:
            color_map.append('green')
        elif node[1]['type'] == 'Dataset' or node[1]['type'] == 'Feature':
            color_map.append('red')
        elif node[1]['type'] == 'Agg' or node[1]['type'] == 'SK_Model':
            color_map.append('blue')
        elif node[1]['type'] == 'SuperNode':
            color_map.append('grey')
        else:
            color_map.append('black')

    nx.draw(ee.graph.graph,
            node_color=color_map,
            pos=pos,
            node_size=100)
    nx.draw_networkx_edge_labels(ee.graph.graph,
                                 pos=pos,
                                 edge_labels={(u, v): d["name"] for u, v, d in ee.graph.graph.edges(data=True)})
    plt.show()

In [2]:
ROOT_DIRECTORY = '../data/home-credit-default-risk'
print(os.listdir(ROOT_DIRECTORY))

['application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [3]:
app_train = ee.load(ROOT_DIRECTORY + '/application_train.csv')
#print('Training data shape: ', app_train.shape().get())
#app_train.head().get()

In [4]:
app_test = ee.load(ROOT_DIRECTORY + '/application_test.csv')

In [16]:
column = 'EXT_SOURCE_1'
source_data = app_train[[column, 'TARGET']][app_train['TARGET'] == 0]
source_data = source_data[app_train[column].notna()]
source_data = source_data[column]

In [17]:
source_data.get()

1         0.311267
6         0.774761
8         0.587334
10        0.319760
11        0.722044
12        0.464831
14        0.721940
15        0.115634
20        0.565655
21        0.437709
25        0.561948
28        0.600396
29        0.297914
31        0.274422
36        0.842763
37        0.804586
48        0.710427
50        0.656225
51        0.311510
52        0.244562
53        0.189596
55        0.552772
57        0.237720
63        0.299606
65        0.286783
66        0.796161
67        0.202477
68        0.411506
70        0.357707
71        0.270766
            ...   
307454    0.351204
307456    0.823222
307458    0.574278
307459    0.122883
307460    0.505346
307461    0.295794
307464    0.676600
307467    0.872456
307468    0.510609
307471    0.623946
307472    0.567361
307473    0.585450
307474    0.511349
307476    0.629572
307478    0.889817
307479    0.407174
307482    0.470808
307485    0.426391
307488    0.479458
307490    0.245623
307491    0.532858
307494    0.

In [11]:
(app_train['EXT_SOURCE_1'].notna()).get() & (app_train['TARGET'] == 0).get()

0         False
1          True
2         False
3         False
4         False
5         False
6          True
7         False
8          True
9         False
10         True
11         True
12         True
13        False
14         True
15         True
16        False
17        False
18        False
19        False
20         True
21         True
22        False
23        False
24        False
25         True
26        False
27        False
28         True
29         True
          ...  
307481    False
307482     True
307483    False
307484    False
307485     True
307486    False
307487    False
307488     True
307489    False
307490     True
307491     True
307492    False
307493    False
307494     True
307495    False
307496     True
307497     True
307498    False
307499     True
307500     True
307501    False
307502    False
307503     True
307504    False
307505    False
307506     True
307507    False
307508     True
307509    False
307510     True
Length: 307511, dtype: b

In [13]:
app_train.get().abs()

TypeError: bad operand type for abs(): 'str'

In [ ]:

class A:
    def __init__(self, a):
        self.a = a
    def b(self):
        print 'hello'
class B(A):
     def __init__(self):
        self.b()

In [ ]:
type([1,2])

In [ ]:
p1 = [1,2,3,4]
p2 = [5,2,3,4]
p3 = [6,3,4]
p = [p1,p2,p3]
s = schedule(p)

In [ ]:
type(s)